In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import input_data

In [5]:
mnist = input_data.read_data_sets('data/',one_hot=True)
trainimg = mnist.train.images
trainlabel = mnist.train.labels
testimg = mnist.test.images[:10]
testlabel = mnist.test.labels[:10]
print('MNIST READY')
print(testimg.shape)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
MNIST READY
(10, 784)


In [6]:
n_input = 784  # 输入层
n_output = 10   # 输出层
# 进行参数的初始化
weights = {
    'wc1':tf.Variable(tf.random_normal([3,3,1,64],stddev=0.1)),
    'wc2':tf.Variable(tf.random_normal([3,3,64,128],stddev=0.1)),
    'wd1':tf.Variable(tf.random_normal([7*7*128,1024],stddev=0.1)),
    'wd2':tf.Variable(tf.random_normal([1024,n_output],stddev=0.1))
}
biases = {
    'bc1':tf.Variable(tf.random_normal([64])),
    'bc2':tf.Variable(tf.random_normal([128])),
    'db1':tf.Variable(tf.random_normal([1024])),
    'bd2':tf.Variable(tf.random_normal([n_output]))
}

In [7]:
# 卷积+池化
def conv_basic(_input,_w,_b,_keepratio):
    # input   转换为 tensorflow支持的格式   四维的 
    _input_r = tf.reshape(_input,shape=[-1,28,28,1]) #  [batch_size,h,w,深度]
    # conv layer 1
    _conv1 = tf.nn.conv2d(_input_r,filter=_w['wc1'],padding='SAME',strides=[1,1,1,1]) #strides[batchsize h w  channel ]     
    # 使用激活函数
    _conv1 = tf.nn.relu(tf.nn.bias_add(_conv1,_b['bc1']))
    # pooling1 层
    _pool1 = tf.nn.max_pool(_conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') # ksize batchsize h w  channels
    _pool_dr1 =  tf.nn.dropout(_pool1,_keepratio) # 输入   保留比例
    # CONV LAYER2
    _conv2 = tf.nn.conv2d(_pool_dr1,_w['wc2'],padding='SAME',strides=[1,1,1,1])
    _conv2 = tf.nn.relu(tf.nn.bias_add(_conv2,_b['bc2']))
    _pool2 = tf.nn.max_pool(_conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    _pool_dr2 = tf.nn.dropout(_pool2,_keepratio)
    # 全连接层
    _dense1 = tf.reshape(_pool_dr2,[-1,_w['wd1'].get_shape().as_list()[0]])
    _fc1 = tf.nn.relu(tf.add(tf.matmul(_dense1,_w['wd1']),_b['db1']))
    _fc1_dr1 = tf.nn.dropout(_fc1,_keepratio)
    # 全连接层2
    _out = tf.add(tf.matmul(_fc1_dr1,_w['wd2']),_b['bd2'])
    
    #return 
    out = {
        'input_r':_input_r,
        'conv1':_conv1,
        'pool1':_pool1,
        'pool1_dr1':_pool_dr1,
        'conv2':_conv2,
        'pool2':_pool2,
        'pool2_dr2':_pool_dr2,
        'dense1':_dense1,
        'fc1':_fc1,
        'fc1_dr1':_fc1_dr1,
        'out':_out
    }
    return out
print('CNN READY')

CNN READY


In [8]:
# 创建X Y 优化  损失
X = tf.placeholder(tf.float32,[None,n_input])
y = tf.placeholder(tf.float32,[None,n_output])
keepratio = tf.placeholder(tf.float32)

# function
_pred = conv_basic(X,_w=weights,_b=biases,_keepratio=keepratio)['out']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=_pred,labels=y))
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
# 求准确率
_corr = tf.equal(tf.argmax(_pred,1),tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(_corr,tf.float32))

init = tf.global_variables_initializer()


# save
save_step = 1  # 选择隔几个epoch进行保存
saver = tf.train.Saver(max_to_keep=3)   # 最多保存3个
print('GRAPH READY')

GRAPH READY


In [15]:
do_train = 2   
train_epochs = 30
batch_size = 16
display_step = 1
path = './log/'
with tf.Session()  as sess:
    sess.run(init)
    writer = tf.summary.FileWriter(path,sess.graph) 
    if do_train==1:
        for epoch in range(train_epochs):
            avg_cost = 0
            #total_batch = int(mnist.train.num_examples/batch_size)
            total_batch = 20
            for i in range(total_batch):
                batch_xs,batch_ys = mnist.train.next_batch(batch_size=batch_size)
                # 开始优化
                sess.run(optm,feed_dict={X:batch_xs,y:batch_ys,keepratio:0.7})
                # 损失值
                avg_cost+=sess.run(cost,feed_dict={X:batch_xs,y:batch_ys,keepratio:1.})
            avg_cost = avg_cost/total_batch

            #打印
            if(epoch%display_step==0):
                print('Epoch:%03d/%03d cost: %.9f'%(epoch,train_epochs,avg_cost))
                train_acc = sess.run(accr,feed_dict={X:batch_xs,y:batch_ys,keepratio:1.})
                print('training accuracy:%.3f'%(train_acc))
    #             feed_test = {X:testimg,y:testlabel,keepratio:1.}
    #             test_acc = sess.run(accr,feed_dict=feed_test)
    #             print('test accuracy:%.3f'%(test_acc))

            if epoch%save_step ==0:
                saver.save(sess,'./save/cnn_mnist_basic.ckpt-'+str(epoch))
        print('OPTIMIZATION FINISHED')
    if do_train==0:
        epoch = train_epochs-1
        saver.restore(sess,'./save/cnn_mnist_basic.ckpt-'+str(epoch))
        test_acc = sess.run(accr,feed_dict={X:testimg,y:testlabel,keepratio:1})
        print('test accuracy:%.3f'%(test_acc))
        


NameError: name 'self' is not defined

In [ ]:
conv_basic
